In [1]:
import os
import sys
import numpy as np
from scipy import sparse
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'svg'
from mpl_toolkits.mplot3d import Axes3D, axes3d
import scipy.cluster.hierarchy as sch
import seaborn as sns
import spacy
import nltk
from konlpy.tag import Okt
import graphviz
from sklearn.utils.fixes import loguniform

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
from sklearn.datasets import load_files

In [4]:
reviews_train = load_files('aclImdb/train/')
text_train, y_train = reviews_train['data'], reviews_train['target']

In [5]:
text_train = [doc.replace(b'<br/>', b'') for doc in text_train]

In [6]:
np.bincount(y_train)

array([12500, 12500], dtype=int64)

In [7]:
reviews_test = load_files('aclImdb/test/')
text_test, y_test = reviews_test['data'], reviews_test['target']
text_test = [doc.replace(b'<br/>', b'') for doc in text_test]

In [8]:
bards_words = ['The fool doth think he is wise,', 'but the wise man knows himself to be a fool']

In [9]:
vect = CountVectorizer().fit(bards_words)
vect.vocabulary_

{'the': 9,
 'fool': 3,
 'doth': 2,
 'think': 10,
 'he': 4,
 'is': 6,
 'wise': 12,
 'but': 1,
 'man': 8,
 'knows': 7,
 'himself': 5,
 'to': 11,
 'be': 0}

In [11]:
bag_of_words = vect.transform(bards_words)
bag_of_words.toarray()

array([[0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1],
       [1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]], dtype=int64)

In [26]:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
X_test = vect.transform(text_test)

In [13]:
vect.get_feature_names_out()

array(['00', '000', '0000000000001', ..., 'østbye', 'über', 'üvegtigris'],
      dtype=object)

In [18]:
[text_train[i] for i in np.argwhere(X_train[:, 9] == 1)[:, 0]][0]

b'"Dutch Schultz", AKA Arthur Fleggenheimer, was a real person and his rather nasty life is fairly well documented. This movie which purports to depict his life should have used a fictional character, because the overdramatized events are too strong a departure from the facts and the chronology. Not only that, it ignores some interesting details which other versions have included such as the public relations fiasco in upstate N.Y. and his religious conversion. It is true that he was executed by Luciano, Lansky, et. al. but that\'s as far as it goes. The exploding plate scene which represents Luciano carrying out the execution of Bo Weinberg in his own home, assisted by his own mother is rediculous. Also, there is the scene in which Dutch approaches his own mother to pay protection to Legs Diamond. It just doesn\'t work. The character of Mrs. Fleggenheimer doesn\'t work either. This movie does not need a doting Jewish mother for comic relief. The lame representation of Legs Diamond was 

In [20]:
# X_train[:, 9] == 1이 (-1, 1) 형태의 shape이기에 argwhere가 [n, 0] 형태로 나타남
np.argwhere(X_train[:, 9] == 1)

array([[  554,     0],
       [ 6583,     0],
       [ 7758,     0],
       [ 9637,     0],
       [10764,     0],
       [15068,     0],
       [16627,     0],
       [17873,     0],
       [22458,     0],
       [23321,     0],
       [23473,     0]])

In [27]:
logreg = LogisticRegression(C=0.1, max_iter=5000).fit(X_train, y_train)
logreg.score(X_test, y_test)

0.87912

In [28]:
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
X_test = vect.transform(text_test)

In [30]:
logreg = LogisticRegression(C=0.1, max_iter=5000).fit(X_train, y_train)
logreg.score(X_test, y_test)

0.87848